In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install --upgrade scipy networkx numpy

1.13.1+cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 23.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 64.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.


In [ ]:
from ast import mod
from torch_geometric.sampler.neighbor_sampler import torch_geometric


import argparse
import os
import torch as th
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.nn import Linear
#from sklearn.model_selection import train_test_split
import numpy as np
from torch_geometric.datasets import TUDataset
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader
from torch_geometric.explain import Explainer, GNNExplainer
import torch_geometric.nn as gnn


class GlobalMeanPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return gnn.global_mean_pool(x, batch)
################################################################################
class IdenticalPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return x

################################################################################
class GCN_plus_GAP(torch.nn.Module):
    def __init__(self, model_name, model_level, input_dim, hidden_dim, output_dim, num_hid_layers, Bias, act_fun, Weight_Initializer, dropout_rate,
                 pred_hidden_dims=[], concat=True, bn=True,
                 add_self=False, args=None):
        if model_name == 'GCN_plus_GAP':
            super(GCN_plus_GAP, self).__init__()
            self.input_dim = input_dim
            print ('GCN_plus_GAP Input_Dimension:', self.input_dim)

            self.hidden_dim = hidden_dim
            print ('GCN_plus_GAP Hidden_Dimension:', self.hidden_dim)

            self.output_dim = output_dim
            print ('GCN_plus_GAP Output_Dimension:', self.output_dim)

            self.num_hid_layers = num_hid_layers
            print ('GCN_plus_GAP Number_of_Hidden_Layers:', self.num_hid_layers)


            self.args = args
            if act_fun == 'ReLu':
                self.act_fun = F.relu
                print('ReLu is Selected.')
            elif act_fun == 'eLu':
                self.act_fun = nn.functional.elu
                print('eLu is Selected.')

            self.GConvs = torch.nn.ModuleList()

            for layer in range(self.num_hid_layers):
                if layer ==0:
                    self.GConvs.append(GCNConv(self.input_dim, self.hidden_dim, bias=Bias))
                else:
                    self.GConvs.append(GCNConv(self.hidden_dim, self.hidden_dim, bias=Bias))

            #print('len(self.GConvs):', len(self.GConvs))
            self.dropout = nn.Dropout(p=dropout_rate)

            if model_level == 'node':
                self.readout = IdenticalPool()
            else:
                self.readout = GlobalMeanPool()

            self.ffn = nn.Linear(self.hidden_dim, self.output_dim, bias=Bias)

            mean = 0
            std = 0.1
            self.initialize_weights(Weight_Initializer, Bias, mean, std)



        else:
            print('This is 2GCN+GAP Model, please type its name well...')

    def initialize_weights(model, Weight_Initializer, Bias, mean, std):
        # 1. Xavier Normal_.  2. Kaiming Normal_.  3. Uniform (0,0.1std)
        if Weight_Initializer == 1:                                             #.      1. Xavier Normal_.
            for i,layers in enumerate(model.children()):
                if isinstance(layers, torch.nn.ModuleList):
                    for j, layer in enumerate(layers.modules()):
                        if isinstance(layer, GCNConv):
                            torch.nn.init.xavier_normal_(layer.lin.weight)
                            if Bias:
                                layer.bias.data.zero_()
                        else:
                            pass
                if isinstance(layers, torch.nn.Linear):
                    torch.nn.init.xavier_normal_(layers.weight)
                    if Bias:
                        layers.bias.data.zero_()

                elif isinstance(layers, (GlobalMeanPool)):
                    pass
                elif isinstance(layers, (IdenticalPool)):
                    pass

        if Weight_Initializer == 2:                                             #.      2. Kaiming Normal_.
            for i,layers in enumerate(model.children()):
                if isinstance(layers, torch.nn.ModuleList):
                    for j, layer in enumerate(layers.modules()):
                        if isinstance(layer, GCNConv):
                            torch.nn.init.kaiming_normal_(layer.lin.weight)
                            if Bias:
                                layer.bias.data.zero_()
                        else:
                            pass
                if isinstance(layers, torch.nn.Linear):
                    torch.nn.init.kaiming_normal_(layers.weight)
                    if Bias:
                        layers.bias.data.zero_()
                elif isinstance(layers, (GlobalMeanPool)):
                    pass
                elif isinstance(layers, (IdenticalPool)):
                    pass

        if Weight_Initializer == 3:                                             #.      3. Uniform (0,0.1std)
            for i,layers in enumerate(model.children()):
                if isinstance(layers, torch.nn.ModuleList):
                    for j, layer in enumerate(layers.modules()):
                        if isinstance(layer, GCNConv):
                            torch.nn.init.normal_(layer.lin.weight.data, mean, std)
                            if Bias:
                                layer.bias.data.zero_()
                        else:
                            pass
                if isinstance(layers, torch.nn.Linear):
                    torch.nn.init.normal_(layers.weight, mean, std)
                    if Bias:
                        layers.bias.data.zero_()
                elif isinstance(layers, (GlobalMeanPool)):
                    pass
                elif isinstance(layers, (IdenticalPool)):
                    pass



    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        Output_of_Hidden_Layers = []
        for i in range(self.num_hid_layers):
            x = self.GConvs[i](x, edge_index)
            x = self.act_fun(x)
            x = self.dropout(x)
            Output_of_Hidden_Layers.append(x)

        pooling_layer_output = self.readout(x, batch)
        ffn_output = self.ffn(pooling_layer_output)
        ffn_output = self.act_fun(ffn_output)

        #log_soft = F.log_softmax(ffn_output, dim=1)
        soft = F.softmax(ffn_output, dim=1)

        return Output_of_Hidden_Layers, pooling_layer_output, ffn_output, soft
        #return Output_of_Hidden_Layers, pooling_layer_output, ffn_output, log_soft, soft

GNN_Model = GCN_plus_GAP(model_name='GCN_plus_GAP', model_level='graph', input_dim=7,
                         hidden_dim=7, output_dim=2, num_hid_layers=2, Bias=True,
                         act_fun='eLu', Weight_Initializer=1, dropout_rate=0.1)
print('===================================================================================')
print(GNN_Model)
print('===================================================================================')







#dataset = TUDataset(root='data/TUDataset', name='MUTAG')

#from torch_geometric.loader import DataLoader
#batched_dataset = DataLoader(dataset, batch_size=4, shuffle=False)

#for batch in batched_dataset:
#    Output_of_Hidden_Layers, pooling_layer_output, ffn_output, soft = GNN_Model(batch)
#    print("ffn_output: ", ffn_output)
#    #print("log_soft: ", log_soft)
#    print("soft: ", soft)
#    break

GCN_plus_GAP Input_Dimension: 7
GCN_plus_GAP Hidden_Dimension: 7
GCN_plus_GAP Output_Dimension: 2
GCN_plus_GAP Number_of_Hidden_Layers: 2
eLu is Selected.
GCN_plus_GAP(
  (GConvs): ModuleList(
    (0): GCNConv(7, 7)
    (1): GCNConv(7, 7)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (readout): GlobalMeanPool()
  (ffn): Linear(in_features=7, out_features=2, bias=True)
)
here:  tensor([[ 0.1848,  0.0123, -0.1787, -0.2115,  0.2106, -0.0936, -0.1079],
        [ 0.1615,  0.0419, -0.1747, -0.1834,  0.2104, -0.0564, -0.0404],
        [ 0.2223, -0.0521, -0.1357, -0.1483,  0.1486, -0.0705, -0.0119],
        [ 0.1658, -0.0018, -0.2813, -0.2062,  0.2606, -0.0891, -0.0789]],
       grad_fn=<DivBackward0>)
ffn_output:  tensor([[-0.0960,  0.2305],
        [-0.1303,  0.1794],
        [-0.2222,  0.2713],
        [-0.1171,  0.3206]], grad_fn=<EluBackward0>)
soft:  tensor([[0.4191, 0.5809],
        [0.4232, 0.5768],
        [0.3791, 0.6209],
        [0.3923, 0.6077]], grad_fn=<SoftmaxBackward0>)
